# Cricsheet Data Exploration and Ingestion
This notebook is meant to provide an overview of Cricsheet data and add it to tables of a SQLite database.

In [1]:
%load_ext sql
%sql sqlite:////Users/tejaskale/Code/cricsheet_data/data/cricsheet.sqlite

In [2]:
import json
import os

### Data exploration

In [3]:
data_dir: str = "./data"
json_files_dir: str = "raw_data"

The raw data downloaded from [Cricsheet](https://cricsheet.org/downloads/#experimental) consists of one JSON file per cricket match. There are 15,933 files available with the data available on January 8, 2024.

In [4]:
num_json_files: int = len(os.listdir(os.path.join(data_dir, json_files_dir)))
num_json_files

15933

In [5]:
json_file_names: list = os.listdir(os.path.join(data_dir, json_files_dir))

In [6]:
sample_json_file_name: str = json_file_names[0]

Every JSON file contains three keys:
- `meta`: metadata containing data version, when the file was created, and revision number.
- `info`: information about the match
- `innings`: a list with one dictionary per innings i.e. a list of 2 for one-day and T20I games and 4 for test matches.
    + Each innings data is a dictionary with the key `team` denoting the batting team and the key `overs` providing ball-by-ball data.

In [7]:
with open(os.path.join(data_dir, json_files_dir, sample_json_file_name), "r") as f:
    sample_json = json.load(f)
sample_json.keys()

dict_keys(['meta', 'info', 'innings'])

In [8]:
sample_json["meta"]

{'data_version': '1.0.0', 'created': '2020-11-16', 'revision': 2}

In [9]:
sample_json["info"].keys()

dict_keys(['balls_per_over', 'city', 'dates', 'event', 'gender', 'match_type', 'missing', 'officials', 'outcome', 'players', 'registry', 'season', 'team_type', 'teams', 'toss', 'venue'])

In [10]:
sample_json["info"]

{'balls_per_over': 6,
 'city': 'Nairobi',
 'dates': ['2010-02-20', '2010-02-21', '2010-02-22', '2010-02-23'],
 'event': {'name': 'ICC Intercontinental Shield'},
 'gender': 'male',
 'match_type': 'MDM',
 'missing': ['player_of_match'],
 'officials': {'match_referees': ['D Govindjee'],
  'reserve_umpires': ['D Angara'],
  'umpires': ['Zameer Haider', 'SR Modi']},
 'outcome': {'winner': 'Kenya', 'by': {'wickets': 5}},
 'players': {'Netherlands': ['AN Kervezee',
   'ES Szwarczynski',
   'B Zuiderent',
   'NA Statham',
   'RN ten Doeschate',
   'Mudassar Bukhari',
   'PW Borren',
   'AF Buurman',
   'MBS Jonkman',
   'RG Nijman',
   'PM Seelaar'],
  'Kenya': ['DO Obuya',
   'RR Patel',
   'CO Obuya',
   'AA Obanda',
   'MA Ouma',
   'JK Kamande',
   'NN Odhiambo',
   'HA Varaiya',
   'E Otieno',
   'NM Odhiambo',
   'JO Ngoche']},
 'registry': {'people': {'AA Obanda': '7081491e',
   'AF Buurman': '03d3dd8e',
   'AN Kervezee': 'dc36a6a5',
   'B Zuiderent': 'd953ed6a',
   'CO Obuya': '5680f1a

In [11]:
len(sample_json["innings"])

4

In [12]:
sample_json["innings"][0]

{'team': 'Netherlands',
 'overs': [{'over': 0,
   'deliveries': [{'batter': 'AN Kervezee',
     'bowler': 'NN Odhiambo',
     'non_striker': 'ES Szwarczynski',
     'runs': {'batter': 1, 'extras': 0, 'total': 1}},
    {'batter': 'ES Szwarczynski',
     'bowler': 'NN Odhiambo',
     'extras': {'legbyes': 1},
     'non_striker': 'AN Kervezee',
     'runs': {'batter': 0, 'extras': 1, 'total': 1}},
    {'batter': 'AN Kervezee',
     'bowler': 'NN Odhiambo',
     'non_striker': 'ES Szwarczynski',
     'runs': {'batter': 0, 'extras': 0, 'total': 0}},
    {'batter': 'AN Kervezee',
     'bowler': 'NN Odhiambo',
     'non_striker': 'ES Szwarczynski',
     'runs': {'batter': 4, 'extras': 0, 'total': 4}},
    {'batter': 'AN Kervezee',
     'bowler': 'NN Odhiambo',
     'non_striker': 'ES Szwarczynski',
     'runs': {'batter': 0, 'extras': 0, 'total': 0}},
    {'batter': 'AN Kervezee',
     'bowler': 'NN Odhiambo',
     'non_striker': 'ES Szwarczynski',
     'runs': {'batter': 0, 'extras': 0, 'tot

## Data modelling

In order to quickly and efficiently retrieve and use the data, it is necessary to store it in a database in order to leverage SQL and other modern database tools that ease analysis. Hence, let us build a pipeline that inserts this data into a OLTP-style database. The tables in this database are designed to avoid redundancy and make it easy to join tables in order to get specific bits of data. First, let us create the tables.

In [12]:
%%sql

create table if not exists team
(
    team_id int primary key,
    name    varchar(20) not null
);

create table if not exists player
(
    player_id int primary key,
    name      varchar(100) not null,
    gender    varchar(1)   not null,
    team_id   int          not null,
    foreign key (team_id) references team (team_id)
);

create table if not exists venue
(
    venue_id int primary key,
    name     varchar(100) not null,
    city     varchar(20)  not null
);

create table if not exists event
(
    event_id int primary key,
    name     varchar(100) not null
);

create table if not exists umpire
(
    umpire_id int primary key,
    name      varchar(100) not null
);

create table if not exists match_type
(
    type_id int primary key,
    name    varchar(10) not null
);

create table if not exists match
(
    match_id int primary key,
    type     varchar(10) not null,
    venue    int         not null,
    event    int         not null,
    toss     int         not null,
    foreign key (type) references match_type (type_id),
    foreign key (venue) references venue(venue_id),
    foreign key (event) references event (event_id),
    foreign key (toss) references team (team_id)
);

create table if not exists dates
(
    match_id   int  not null,
    match_date date not null,
    foreign key (match_id) references match (match_id)
);

create table if not exists officiates
(
    match_id  int not null,
    umpire_id int not null,
    foreign key (match_id) references match (match_id),
    foreign key (umpire_id) references umpire (umpire_id)
);

create table if not exists team_plays
(
    match_id int not null,
    team_id  int not null,
    foreign key (match_id) references match (match_id),
    foreign key (team_id) references team (team_id)
);

create table if not exists player_plays
(
    match_id  int not null,
    player_id int not null,
    foreign key (match_id) references match (match_id),
    foreign key (player_id) references player (player_id)
);

create table if not exists extras
(
    extras_id    int primary key,
    name         varchar(20) not null,
    has_free_hit bool        not null
);

create table if not exists ball
(
    match_id        int not null,
    innings         int not null,
    over            int not null,
    ball            int not null,
    batting_team    int not null,
    bowling_team    int not null,
    striker         int not null,
    non_striker     int not null,
    bowler          int not null,
    non_extras_runs int not null,
    extras_runs     int not null,
    extras          int,
    total_runs      int not null,
    foreign key (match_id) references match (match_id),
    foreign key (batting_team) references team (team_id),
    foreign key (bowling_team) references team (team_id),
    foreign key (striker) references player (player_id),
    foreign key (non_striker) references player (player_id),
    foreign key (bowler) references player (player_id),
    foreign key (extras) references extras (extras_id)
);


In [13]:
match_id: int = int(sample_json_file_name.split(".")[0])
match_id

438938

In [14]:
def get_match_id(json_file_name: str) -> int:
    """
    Get match ID from JSON file name.
    
    :param json_file_name: Name of the JSON file. 
    :return: Match ID as integer.
    """
    return int(json_file_name.split(".")[0])    

In [15]:
def load_match_json(json_file_path: str) -> dict:
    """
    Load a JSON file and return it as a dictionary.

    :param json_file_path: Path to the JSON file.
    :return: The loaded JSON as a dictionary.
    """
    with open(json_file_path, "r") as json_file:
        match_data = json.load(json_file)
    return match_data